In [ ]:
import tabula
import pandas as pd
import re
import numpy as np
import math

In [ ]:
file_path = "Downloads/woo-used-voucher-codes-2021-01-11 (1).pdf"
page_num = 264

In [ ]:

df = pd.DataFrame([])
df_combine = pd.DataFrame([])
for pageiter in range(page_num):
#for pageiter in [0,1,262,263]:
    print(pageiter)
    df = tabula.read_pdf( file_path, pages = pageiter+1, guess=False)[0]
    df_combine = pd.concat([ df_combine, df]) #again you can choose between merge or concat as per your need
    #if pageiter > 3:
    #    break

In [ ]:
df_combine = df_combine.reset_index().drop('index', axis = 1)
df_combine

In [ ]:
for index, row in df_combine.iterrows():
    #print(row['Voucher Code'])
    if type(row['Voucher Code']) == str:
        print(index)
        current = row['Voucher Code']
        #df_combine['Voucher Code'][index] = current
    else:
        df_combine.at[index, 'Voucher Code'] = current

In [ ]:
df_output = pd.DataFrame()

codes = pd.unique(df_combine['Voucher Code'])
for code in codes:
    df_tmp = df_combine[df_combine['Voucher Code'] == code]
    
    # Get SKU
    df_sku = df_tmp['Product Information'] 
    df_sku = df_sku.str.replace('\r', ' ').str.split('SKU:').str[1]
    sku = df_sku.iloc[0]
    # check if nan
    if sku != sku:
        #print(sku)
        df_sku = df_tmp[df_tmp['Product Information'].str.contains('SKU:', na = False)]
        try:
            sku = df_sku.iloc[0]['Product Information'].replace('SKU: ', '')
        except:
            sku = 'error'
    print(sku)
    
    name = ''
    # Finn navn
    df_name = df_tmp['Product Information'] 
    df_name = df_name.str.replace('\r', ' ')
    for val in df_name:
        #print(val)
        if 'Name:' in val and 'Price:' in val:
            #print('first page shit')
            name = val.split('Name: ')[1].split(' Price:')[0]
        else:
            if 'Price:' in val:
                break
            else:
                name += val.replace('Name: ', '')
            #print('not first page shit')
    
    #name = name.replace('(Trondheim)(Trondheim)' , '(Trondheim)')
    
    # Parse pris
    pris = df_tmp[df_tmp['Product Information'].str.contains("Price:", na = False)]
    pris = pris['Product Information'].str.split("Price:").str[-1]
    pris = pris.str.split("\r").str[0]
    try:
        pris = pris.iloc[0]
        pris = float(pris.split("kr ")[1].replace('.','').replace(',','.'))
    except:
        pris = 'error'
    
    df_new = pd.DataFrame([[code, sku, name, pris]])
    df_output = pd.concat([df_output, df_new])

In [ ]:
df_output.columns = ['Kode', 'Sku', 'Kursnavn', 'Pris']
df_output = df_output.reset_index().drop('index', axis = 1)


In [ ]:
df_output

In [ ]:
df_oppsummering = df_output.groupby('Sku').sum().drop(['Kode', 'Kursnavn'], axis = 1).reset_index()
df_oppsummering.columns = ['Sku', 'Sum']
df_oppsummering
df_oppsummering = df_oppsummering.merge(df_output.drop('Kode', axis = 1).drop_duplicates('Sku'), how = 'inner', on = 'Sku')
df_oppsummering.to_csv('redeemed_used_vouchers_2020.csv', sep=";")

In [ ]:
df_oppsummering

In [ ]:
df_oppsummering.iloc[:-1]['Sum'].sum()